In [2]:

from obspy import Stream , Trace
from obspy.taup import TauPyModel
from geopy.distance import geodesic

import numpy as np
import glob
import os
import time
import DasPrep as dp
import datetime
import obspy
from scipy import signal
import pandas as pd

from geopy.distance import geodesic




def  get_p_and_s_arrival_times(source_depth_in_km, distance):
    """
    **Theoretical p&s wave arrival time**
    reference page:https://docs.obspy.org/packages/obspy.taup.html
    Parameters：
    - model: 地震模型，如 "ak135" 或 "prem"
    - source_depth_in_km: event deepth（单位：千米）
    - distance_in_degree: epicenter（unit：degree）--use epicenter(km)/111
    Return：
    - P wave arrival time---travel_time_p,travel_time_s
    - S wave arrival time-travel_time_s
    """
    distance_in_degree = distance / 111

    # 创建 TauPyModel 实例
    model = TauPyModel(model="iasp91")
    # 获取 P 波到时
    travel_time_p ,travel_time_s = False , False
    try:
        arrival_p = model.get_travel_times(source_depth_in_km=source_depth_in_km,
                                           distance_in_degree=distance_in_degree,
                                           phase_list=["P", 'p'])

    except ValueError:
        arrival_p = model.get_travel_times(source_depth_in_km=source_depth_in_km,
                                           distance_in_degree=distance_in_degree,
                                           phase_list=["p"])
    if len(arrival_p)> 0:
        travel_time_p = round(arrival_p[0].time, 3)

    # 获取 S 波到时
    try:
        arrival_s = model.get_travel_times(source_depth_in_km=source_depth_in_km,
                                           distance_in_degree=distance_in_degree,
                                           phase_list=["S", 's'])
    except ValueError:
        arrival_s = model.get_travel_times(source_depth_in_km=source_depth_in_km,
                                           distance_in_degree=distance_in_degree,
                                           phase_list=["s"])

    if len(arrival_s)> 0:
        travel_time_s = round(arrival_s[0].time, 3)

    return travel_time_p, travel_time_s


theory_arrival_cat = pd.DataFrame(columns = ["travel_time_p_date", "travel_time_s_date", 'travel_time_p', 'travel_time_s' , 'Date','Time','Lat.','Lon.','Dep.','Mag.','num','station','has'])

eq_cat = pd.read_csv("/home/disk/disk01/wzm/DAS_DL_Dataset/src/202ML0_guangdongCatalog.eqt", delim_whitespace=True , encoding='gb18030' ,header=None, names=['Date','Time','Lat.','Lon.','Dep.','Mag.','num','station'] , dtype = str)

print(str(eq_cat['Date'].values[0])+' '+str(eq_cat['Time'].values[0]))
eq_time = np.array([datetime.datetime.strptime(str(eq_cat['Date'].values[i])+' '+str(eq_cat['Time'][i]), '%Y%m%d %H%M%S.%f') - datetime.timedelta(hours=8)
       for i in range(len(eq_cat))])

start_time , end_time = datetime.datetime(2024,9,21) , datetime.datetime(2024,10,21,2,19,40,357)
has_eq_window =  np.where((eq_time > start_time) & (eq_time < end_time))
# print(has_eq_window)
has_eq_cat = eq_cat.loc[has_eq_window]
# print(has_eq_cat)
for idx , event in has_eq_cat.iterrows():
    print(idx, event)
    # 定义两个坐标点
    fiber_0 = (23.929621 , 114.456382	)
    event_loc = (float(event['Lat.']), float(event['Lon.']))

    # 计算距离
    distance = geodesic(fiber_0, event_loc).km

    travel_time_p, travel_time_s = get_p_and_s_arrival_times( float(event['Dep.']),distance)
    if travel_time_p!=False : 
        print( travel_time_p, travel_time_s )
        travel_time_p_date = eq_time[idx] + datetime.timedelta(seconds=travel_time_p)
        travel_time_s_date = eq_time[idx] + datetime.timedelta(seconds=travel_time_s)
        theory_arrival_cat.loc[len(theory_arrival_cat)] = [travel_time_p_date, travel_time_s_date , travel_time_p, travel_time_s ,   event['Date'],event['Time'],event['Lat.'],event['Lon.'],event['Dep.'],event['Mag.'],event['num'],event['station'] , True]
    else:
        theory_arrival_cat.loc[len(theory_arrival_cat)] = [eq_time[idx] , eq_time[idx]  , eq_time[idx] , eq_time[idx]  ,   event['Date'],event['Time'],event['Lat.'],event['Lon.'],event['Dep.'],event['Mag.'],event['num'],event['station'] , False]


theory_arrival_cat.to_csv("/home/disk/disk01/wzm/DAS_DL_Dataset/data/xfj/theory_arrival_cat_Pp.csv",index=False,sep=',')


20200101 024917.00
18512 Date              20240921
Time             092816.00
Lat.             23.318300
Lon.            117.138000
Dep.                   0.8
Mag.                 -1.00
num                      0
station    广东南澎岛台(S-P2.2s)
Name: 18512, dtype: object
42.351 74.923
18513 Date              20240921
Time             093553.00
Lat.             23.057199
Lon.            117.208000
Dep.                   0.9
Mag.                 -1.00
num                      0
station    广东南澎岛台(S-P3.0s)
Name: 18513, dtype: object
44.236 78.314
18514 Date              20240921
Time             102228.00
Lat.             23.195801
Lon.            117.356003
Dep.                   0.6
Mag.                 -1.00
num                      0
station    广东南澎岛台(S-P1.6s)
Name: 18514, dtype: object
45.483 80.551
18515 Date         20240921
Time        113902.00
Lat.        23.731001
Lon.       114.620003
Dep.              1.2
Mag.             8.69
num                 0
station          广东河源
Name: 1851